# JEM092 Asset Pricing - Homework 2

### Group 82780095

Petr Dvořáček, Jan Kubal, Matyáš Mattanelli

---

<a id='Contents'></a> 
## List of Contents

- [Abstract](#abstract)
- [1. Introduction](#introduction)
- [2. Data Description](#data)
- [3. Empirical Methodology](#methodology)
- [4. Results](#results)
    - [Univariate sort](#univariate)
    - [Bivariate sort](#bivariate)
    - [Fama-MacBeth regression](#fm_regression)
- [5. Conclusion](#conclusion)
- [Appendix](#appendix)
    - [Appendix A](#appendix_a)
    - [Appendix B](#appendix_b)

## Abstract
<a id='abstract'></a> 

The aim of this paper is to investigate the relationship between three sorting variables Beta, Size, and Skewness and the outcome variable one-ahead excess returns. For this purpose it uses a sample of 250 stocks' one-ahead monthly excess returns and performs univariate and bivariate portfolio sort analysis, and Fama-MacBeth regression. Findings of the univariate portoflio sort analysis are mostly in line with the extant literature - insignificant relationship between the Beta as well as the (total) Skewness and one-ahead excess returns for both equally-weighted and value-weighted portfolios, statistically significant and negative relationship of the Size and excess returns also for both portfolios. Then, portfolios were sorted by two variables - firstly, by Size and Beta, and secondly, by Size and Skewness. The former seems to show consistently negative and significant effect of Size on returns across various levels of Beta, i. e. Size difference portfolio has negative mean and risk-adjusted values. On the other hand, the Beta difference portfolio is insignificant across all levels of Size, therefore, we conclude that the significant relationship between Size and returns cannot be explained by different values of Beta. The latter way of sorting shows constant relationship between Size and returns for various levels of Skewness (and vice versa), as the effect of the difference-in-difference portfolio was insignificant. Lastly, the Fama-MacBeth regression analysis considering solely the relationship between Skewness and one-ahead excess returns does not provide any evidence of a statistically significant relationship. When Size and Beta are added into the regression as explanatory variables, we find that only Size has a statistically significant and negative effect.

## 1. Introduction
<a id='introduction'></a> 

The aim of this paper is to investigate the relationship between three sorting variables Beta, Size, and Skewness and the outcome variable one-ahead excess returns. For this purpose it utilizes various market data such as the Fama-French 3 factors, Momentum, and Liquidity factor, and firm level data such as daily adjusted close prices and Market Capitalization. The focus is on sorting stocks into various portfolios. Furthermore, it examines time-series properties of the formed portfolios, examines cross-sectional variation in portfolios, and comments on importance of the sorting variables. In addition, the Fama-French-Carhart-Pastor-Stambaugh factor model is utilized to risk-adjust the results. Lastly, the Fama-MacBeth regression is performed.

The paper uses a sample of 250 stocks' one-ahead monthly excess returns from the begginning of January 2007 to the end February 2022. In the Data Description part of this paper, an overview containing summary statistics and pairwise correlations of the variables is provided along with the persistence analysis. Then, the main steps regarding the univariate and bivariate portfolio sort analysis and Fama-MacBeth regression are thoroughly disccussed in the Empirical Methodology part. The key results are presented in the Results part of this paper. The last part summarizes the results and compares them to the existing literature, most importantly, the textbook by Bali *et al.* (2016). Additional notes and steps used in the analysis (Data preparation and List of stocks - their tickers) can be found in the Appendix.

## 2. Data Description
<a id='data'></a> 

In this section we provide an overview of the utilized data. We examine summary statistics of the variables as well as pairwise correlations. Lastly, we perform a persistence analysis. We start by loading the prepared data along with the necessary packages. The steps we took during data preprocessing and variable derivations are available in the Appendix.

In [1]:
#Loading the packages
#install.packages("xts", "readr", "lubridate", "quantmod", "moments", "sandwich", "lmtest")
suppressPackageStartupMessages({
    library(xts) #Handling xts objects
    library(readr) #Reading csv files
    library(lubridate) #Date manipulation
    library(quantmod) #Returns calculation
    library(moments) #Skewness and kurtosis functions
    library(sandwich) #Robust covariance matrix estimation
    library(lmtest) #Robust covariance matrix estimation
})

#Loading the data
monthly_returns <- readRDS("Final data/monthly_excess_returns.RData")
skewness_data <- readRDS("Final data/skewness.RData")
betas <- readRDS("Final data/betas.RData")
market_cap <- readRDS("Final data/market_cap_monthly.RData")
size <- readRDS("Final data/size.RData")
fffactors <- readRDS("Final data/fffactors_monthly.RData")
momentum <- readRDS("Final data/momentum.RData")
liquidity <- readRDS("Final data/liquidity.RData")

Before proceeding to the summary statistics, we need to take a look at the inspected period. The required time span according to the first homework is 01.01.2007 - 28.02.2022. However, in the second assignment the required period is 01.01.2007 - 31.12.2020. We select the former in order to utilize as much data as possible. In addition, the data for the Market Capitalization variable are available from 28.02.2009 onwards which decreases the usable time span. Furthermore, the liquidity measure ends in December 2021 which shrinks our data even further. Nevertheless, for the purpose of data description and also for Beta and Skewness calculation, we retain all values for all variables available after 01.01.2007 and until 28.02.2022. We will make the proper adjustments later in the empirical analysis.

In [2]:
#Restricting the period (01/01/2007 - 28/02/2022)
market_cap <- market_cap["/2022-02-28"]
size <- size["/2022-02-28"]
fffactors <- fffactors["/2022-02-28"]
momentum <- momentum["/2022-02-28"]

The last step we take before inspecting the patterns in data is the description of the utilized variables. The outcome variable in our analysis are the one-ahead monthly excess returns ($r_{t+1}$). We calculate the excess returns from the Adjust Close prices ($p_t$) and risk-free rate ($r_f$) as $r_{t+1}=\frac{p_{t+1}-p_t}{p_t} - r_f$. As for the sorting variables, we inspect the effect of Size, Beta, and Skewness. The first one is calculated simply as the natural logarithm of Market Capitalization. Beta, the second measure representing market sensitivity, is derived by regressing the daily returns from past 12 months on the market premium and storing the estimated coefficient ($\beta$). Therefore, for each monthly period $t$ we estimate $r_k=\alpha+\beta Mkt_k + \epsilon_k$, where $r_k$ is the daily return at day $k$, $\alpha$ is the intercept, $\beta$ is the value of interest, $Mkt_k$ is the market premium in day $k$ and $\epsilon_k\sim N(0,\sigma^2)$. The last $k$ data point is the last day of month $t$. Lastly, we inspect the effect of Skewness. Generally, three measures of skewness are common in the literature: Total skewness, Co-skewness, and idiosyncratic skewness. In our analysis we consider only the first one which is calculated as $Skew_t=\frac{\frac{1}{n}\sum_{k=1}^n(r_k-\bar{r_n})^3}{[\frac{1}{n}\sum_{k=1}^n(r_k-\bar{r_n})^2]^{\frac{3}{2}}}$, where $t$ denotes the month, $k$ denotes the day, $n$ is the number of days used for the calculation (12 months back from $t$), and $\bar{r_n}$ is the average return over the $n$ observations. Each of the measures are calculated for each month and each stock in our data set. The computations are available in the Appendix.

Finally, we can proceed with describing the data. Our final data set contains monthly observations for 250 stocks in the period between 01.01.2007 and 28.02.2022. Since we utilize Market Capitalization as the weights in value-weighted portfolios, we provide its summary statistics as well. For each variable and each period we calculate 12 statistics in total. We then take an average of these values over all available periods. In order to facilitate the process, we define a custom function.

In [3]:
#Defining a function to calculate the summary statistics
calc_sum_stats <- function(var_data) {
  period_stats <- matrix(ncol = 12, nrow = nrow(var_data)) #A matrix for the statistics from each period
  perc_5 <- function(x, na.rm) {return(quantile(x, probs = 0.05, na.rm = na.rm))} #Function to calculate the 5th percentile of data
  perc_25 <- function(x, na.rm) {return(quantile(x, probs = 0.25, na.rm = na.rm))} #Function to calculate the 25th percentile of data
  perc_75 <- function(x, na.rm) {return(quantile(x, probs = 0.75, na.rm = na.rm))} #Function to calculate the 75th percentile of data
  perc_95 <- function(x, na.rm) {return(quantile(x, probs = 0.95, na.rm = na.rm))} #Function to calculate the 95th percentile of data
  calc_n <- function(x, na.rm) {return(sum(!is.na(x)))} #Function to calculate the number of observations
  funcs <- list(mean, sd, skewness, kurtosis, min, perc_5, perc_25, median, perc_75, perc_95, max, calc_n) #Store the functions we will use
  for (i in 1:nrow(var_data)) { #Loop through the periods
    for (j in 1:length(funcs)) { #Loop through the functions
      period_stats[i, j] <- funcs[[j]](as.numeric(var_data[i, ]), na.rm = T) #Apply each function and store the result 
    }
  }
  time_avg <- apply(period_stats, 2, mean, na.rm = T) #Take a time series mean of each statistic
  return(round(time_avg, 2)) #Round to two decimals and return the result
}

#Applying the function
sum_stats <- as.data.frame(matrix(nrow = 5, ncol = 12)) #Place holder for the final summary statistics
colnames(sum_stats) <- c("Mean", "Sd", "Skewness", "Kurtosis", "Min", 
                          "5%", "25%", "Median", "75%", "95%", "Max", "n") #Rename the columns as the statistics
var_names <- c("One-ahead Returns", "Skewness", "Beta", "Size", "Market cap") #Define a vector of variable names
row.names(sum_stats) <- var_names #Rename the row names as the variables
data_list <- list(lag(monthly_returns, -1)[1:(nrow(monthly_returns) - 1),],
                 skewness_data, betas, size, market_cap) #Store the data in a list for looping (shift monthly returns forward to get one-ahed returns)
for (i in 1:5) { #Loop through the variables and apply the function
  sum_stats[i, ] <- calc_sum_stats(data_list[[i]])
}

The resulting summary statistics are available below. Each row shows the 12 statistics for each of the 5 variables. The following statistics are computed and discussed: mean, standard deviation, skewness, kurtosis, minimum, 5%, 25%, 50% (median), 75% and 95% percentile, maximum and number of non-missing observations. A brief glance at the first row reveals that the average excess return in our data set is positive. Both the mean and median are identical suggesting a symetric distribution. This is confirmed by the skewness (first row, third column) which is quite close to zero. The value is slightly positive which likely reflects that fact that there is a lower bound on returns (-100%) but no upper bound. The value for kurtosis on the other hand is well above 3 (kurtosis of normal distribution) and therefore, the distribution of the returns is likely heavy-tailed. The maximum and minimum excess returns are 35% and -23%, respectively. The maximum value, however, is likely an outlier given the relatively low value of the 95th percentile. 

As for the Skewness sorting variable, it seems to be distributed symmetrically around zero. This confirms our previous observations that the returns are not substantially skewed. It does attain some extreme values at both tails.

A second sorting variable of interest is the stocks' sensitivity to market. As we can see, the average value is around one which indicates that on average the excess returns in our dataset almost perfectly mimic the market portfolio. In addition, it does not attain a single negative value suggesting that all the stocks react positively to the market. This may have important implications for our analysis since the considered group of stocks may not be entirely representative of the whole population. The distribution of Betas seem to be quite close to normal.

On the other hand, Market Capitalization seems to be very far from normal distribution. However, the logarithmic transformation seems to remedy this issue since the Size variable is quite close to normal distribution in terms of both skewness and kurtosis. The positive skewness of Market Capitalization is caused by some large outliers in the upper tail which signals that there are one or a few firms that cover a substantially larger portion of the market than the rest. The maximum value is very far from the 95% percentile. Once again, Size does not seem to suffer from these issues since the logarithmic tranasformation decreases the emphasis of outliers.

Overall, we can see that in the average period we have majority of the observations for all stocks (denoted by the last column) with the exception of the Market Capitalization and Size variables which seem to have slightly more missing observations. 

In [4]:
#Summary statistics
sum_stats

Mean  Sd    Skewness Kurtosis Min   5%    25%   Median 75%  
One-ahead Returns  0.01  0.07 0.50      7.42    -0.23 -0.09 -0.03  0.01   0.05
Skewness           0.02  0.97 0.25     18.29    -4.92 -1.22 -0.35 -0.03   0.34
Beta               1.02  0.37 0.39      3.34     0.21  0.45  0.76  1.00   1.25
Size               2.99  1.12 0.60      3.24     0.43  1.47  2.18  2.84   3.63
Market cap        45.48 90.88 5.21     38.28     1.82  5.06  9.77 18.46  40.71
                  95%    Max    n     
One-ahead Returns   0.13   0.35 248.87
Skewness            1.45   5.41 248.64
Beta                1.65   2.24 248.64
Size                5.08   6.56 244.67
Market cap        168.12 858.28 245.14

Moving on from the summary statistic, we provide the average pairwise correlations between the variables in question. Similarly as in the previous case, we firstly calculate the cross-sectional correlations between the variables and then average the values across periods.

In [5]:
#Defining a function to calculate the average correlation between two variables
calc_corr <- function(var1, var2, method = "pearson") { #Expects data for both variables and an indicator for type (default is pearson)
  var1 <- var1[index(var1) %in% index(var2)] #Filter only data available for both variables
  cross_sec_corrs <- rep(NA, nrow(var1)) #Empty vector the results of each period
  for (i in 1:nrow(var1)) { #Loop through the periods available for the first variable
    cross_sec_corrs[i] <- cor(as.numeric(var1[i]), as.numeric(var2[index(var1)[i]]),
                               use = "pairwise.complete.obs", method = method) #Calculate the correlation for each period
  }
  return(round(mean(cross_sec_corrs, na.rm = T), 2)) #Return the rounded time series average of the correlations
}

#Applying the function
corrs <- as.data.frame(matrix(nrow = 5, ncol = 5)) #Empty matrix for the results
colnames(corrs) <- var_names #Column names
row.names(corrs) <- var_names #Row names
indices <- combn(1:5, 2) #Get the indices for each pair (unique and excluding pairing variables with themselves)
for (i in 1:ncol(indices)) { #Loop through the pairs
  corrs[indices[1, i], indices[2, i]] <- calc_corr(data_list[[indices[1, i]]],
                                                   data_list[[indices[2, i]]], method = "spearman") #Spearman correlation in the upper triangle

  corrs[indices[2, i], indices[1, i]] <- calc_corr(data_list[[indices[2, i]]],
                                                   data_list[[indices[1, i]]]) #Pearson correlation in the lower triangle
}

The results are available in the table below. The diagonal contains missing values since the correlation of a variable with itself is 1 and as such is not relevant. Above the diagonal we present the Spearman correlation while below is the Pearson correlation. As can be seen, the values are largely similar with the exception of Size and Market Capitalization since the Spearman correlation captures the perfectly monotonic relationship while Pearson correlation shows only the degree of linear dependence. As for the remaining values, we can see that the correlations are very low. Nevertheless, disregarding the magnitude, we can observe that returns are positively correlated with Beta as predicted by the CAPM and negatively correlated with Size and Market Capitalization which is a common empirical observation. The Skewness measure is essentially uncorrelated with Returns. As Bali *et al.* (2016) indicated, this might be due to the fact that the 12 month daily measure of total skewness may not be an appropriate proxy for true skewness due to its noisiness. We will see later how it affects our analysis. The largest in magnitude is the Pearson correlation between Beta and Size which indicates that larger stocks in our data set are less market-sensitive. Bali *et al.* (2016) found in fact a positive correlation between these variables which raises a question regarding the relevance of the subset of stocks utilized in our analysis. As pointed out above, the value of Beta is strictly positive which may be the cause of the found relationship. Lastly, Skewness appears to be positively correlated with Beta. This contradicts the findings of Bali *et al.* (2016) as well.

In [6]:
#Pairwise correlations
corrs

One-ahead Returns Skewness Beta  Size  Market cap
One-ahead Returns    NA             0.00      0.03 -0.06 -0.06     
Skewness           0.00               NA      0.14  0.01  0.01     
Beta               0.04             0.11        NA -0.13 -0.13     
Size              -0.07             0.02     -0.14    NA  1.00     
Market cap        -0.02             0.03     -0.08  0.75    NA

Lastly, we perform the persistence analysis. We consider the 1st, 3rd, 6th, 12th, 24th, and 36th lag for each variable. The reason why we use the 12th, 24th, and 36th lags is to avoid inspecting mechanical persistence caused by the overlapping calculation periods for Beta and Skewness. We present the average autocorrelation over all periods for a given lag for each variable.

In [7]:
#Defining a function to calculate persistence of a variable
calc_pers <- function(var_data, lags = c(1, 3, 6, 12, 24, 36)) { #Expects an xts object and the lags for which to calculate the correlations
  final_persist <- rep(NA, length(lags)) #A vector to store the results 
  for (i in 1:length(lags)) { #Loop through the lags
    cross_persist <- rep(NA, nrow(var_data)) #A vector to store the results from each period
    for (j in 1:nrow(var_data)) { #Loop through the periods
      current_period <- index(var_data)[j] #Store the current period
      lagged_period <- as.Date(as.yearmon(current_period %m-% months(lags[i])), frac = 1) #Derive the lagged period
      if (!lagged_period %in% index(var_data)) { #If the lag is unavailable, skip
        next
      }
      cross_persist[j] <- cor(as.numeric(var_data[j]),
                              as.numeric(var_data[lagged_period]),
                              use = "pairwise.complete.obs") #Calculate Pearson between the variable and the appropriate lag
    }
    final_persist[i] <- mean(cross_persist, na.rm = T) #Calculate the time series mean
  }
  return(round(final_persist, 3)) #Return the rounded correlations
}

#Apply the function
lags <- c(1, 3, 6, 12, 24, 36) #Specify the required lags
persistence <- as.data.frame(matrix(ncol = length(lags), nrow = 5)) #Data frame for the results (lags in columns, vars in rows)
colnames(persistence) <- paste("Lag ", lags) #Rename the columns with lag numbers
row.names(persistence) <- var_names #Rename the rows with variable names
for (i in 1:nrow(persistence)) { #Loop through the variables and calculate their persistence
  persistence[i, ] <- calc_pers(data_list[[i]], lags = lags) #Apply the function
}

The results of the table below seem to follow the findings of Bali *et al.* (2016). The Returns do not show any substantial autocorrelations which demostrates their unpredictable behavior. Similarly, the Skewness variable does not appear to be persistent. None of the 12th, 24th or 36th lags exhibit high correlation. The above-mentioned mechanical persitence is demonstrated in the 1st, 3rd, and 6th lags where the correlations are quite high. Since persistence is a desirable property of a sorting variable, the suitability of our proxy for Skewness is further undermined. On the other hand, Beta, Size, and Market Cap all appear to be strongly pesistent. The autocorrelations do not substantially decay even in further lags. This suggests that the market sensitivity of a stock does not change much over time on average and the same applies to its market share. 

In [8]:
#Persistence analysis
persistence

Lag  1 Lag  3 Lag  6 Lag  12 Lag  24 Lag  36
One-ahead Returns 0.000  -0.023 0.002  -0.005  -0.011  0.011  
Skewness          0.906   0.721 0.489   0.039   0.048  0.012  
Beta              0.988   0.951 0.887   0.768   0.671  0.641  
Size              0.998   0.994 0.988   0.975   0.951  0.929  
Market cap        0.998   0.994 0.987   0.975   0.956  0.935

## 3. Empirical Methodology
<a id='methodology'></a> 

This section presents an overview of the utilized methodology. We also introduce the functions used to perform the analyses. These include the univariate portfolio analysis, bivariate portfolio analysis, and Fama-MacBeth regression.

The univariate portfolio analysis inspects the relationship between a single indenpendent variable and a dependent variable. The dependent variable in our case are the one-ahead returns. We separately investigate three independent variables: Beta, Size, and Skewness. The nature of the analysis allows us to not make any assumptions regarding the linearity of the relationship.

For each variable and each period we divide the stocks into 5 portfolios. These are formed based on breakpoints calculated as the 20th, 40%, 60%, and 80% percentile of the sorting variable. When allocating the stocks into the portfolios we consider closed intervals to prevent the existence of empty portfolios. This may result in some stocks being allocated to multiple portfolios. This approach follows the textbook by Bali *et al.* (2016). For each portfolio we then calculate the cross-sectional average of the sorting variable as well as the outcome variable. We consider both an equally weighted average as well as value-weighted average. Subsequently, we calculate the difference portfolio as the difference between the average values of the last and first portfolios.

After obtaining the cross-sectional averages for each period, we calculate the time-series average for each portfolio. We report the *t*-statistic associated with the values adjusted using the robust Newey-West covariance matrix with 6 lags to ensure robustness to serial correlation. In addition, we use five factors to obtain Fama-French-Carhart-Pastor-Stambaugh alpha and its *t*-statistic. The five factors include the Market premium, High Minus Low, Small Minus Big, Momentum, and Liquidity. The value of alpha represents the risk-adjusted returns.

In [9]:
#Defining a function to perform a univariate sort
univariate_sort <- function(sort_variable, no_of_ports = 5, weighted = F) {
  sort_variable_name <- gsub("MDLZ.", "", colnames(sort_variable)[1]) #Extracting the name of the sorting variable
  final_ouptut <- as.data.frame(matrix(ncol = no_of_ports + 1, nrow = 5)) #Data frame for the final output
  colnames(final_ouptut) <- c(1:no_of_ports, paste(no_of_ports, 1, sep = "-")) #Column names for clarity
  row.names(final_ouptut) <- c(paste0("Mean of ", sort_variable_name), "Mean of one-ahead returns", "t-statistic", "Alpha", "t-statistic (alpha)")
  all_avg_sort_values <- c() #Empty vector to which we will append the results from each period (sort variable)
  all_avg_returns <- c() #Empty vector to which we will append the results from each period (returns)
  for (i in 1:nrow(sort_variable)) { #Loop through the rows (periods)
    current_period <- index(sort_variable)[i] #Store the current period
    next_period <- as.Date(as.yearmon(current_period %m+% months(1)), frac = 1) #Store the next period (for the 1-ahead returns)
    if (!next_period %in% index(monthly_returns)) { #In case we do not have 1-ahead returns, skip the period
      next
    }
    average_sort_values <- rep(NA, no_of_ports) #Place holder for the cross-sectional average values of the sort variable
    average_returns <- xts(matrix(nrow = 1, ncol = no_of_ports + 1),
                           order.by = current_period) #Place holder for the cross-sectional average values of the 1-ahead returns
    breakpoints <- quantile(sort_variable[current_period], probs = seq(1/no_of_ports, 1 - 1/no_of_ports, 1/no_of_ports), na.rm = T) #Find the breakpoints
    for (j in 1:no_of_ports) { #Looping through the portfolios
      if (j == 1) { #For the first portfolio we have only one condition
        indic <- which(sort_variable[current_period] <= breakpoints[j]) #Find the stocks in the portfolio and save their column indices
      } else if (j == no_of_ports) { #For the last portfolio we have only one condition as well
        indic <- which(sort_variable[current_period] >= breakpoints[j -  1])
      } else { #The rest of the portfolios (two conditions)
        indic <- which(sort_variable[current_period] >= breakpoints[j - 1] &
                       sort_variable[current_period] <= breakpoints[j]) #"=" at both inequalities to prevent empty portfolios
      }
      if (weighted == T) { #Market cap weighted average
        mc_weights <- as.numeric(market_cap[current_period, indic]) #Storing the weights
        if (length(mc_weights) == 0) { #If we do not have the market cap data, we have to skip
          next
        }
        mc_weights[is.na(mc_weights)] <- 0 #0 value for missings = they have no weight
        average_returns[, j] <- weighted.mean(as.numeric(monthly_returns[next_period, indic]), w = mc_weights, na.rm = T)
      } else { #Equally-weighted average
        average_returns[, j] <- mean(monthly_returns[next_period, indic], na.rm = T) #Calculate the average of 1-ahead returns
      }
      average_sort_values[j] <- mean(sort_variable[current_period, indic], na.rm = T) #Calculate the average of the sort variable for the given portfolio
    }
    average_returns[, no_of_ports + 1] <- average_returns[, no_of_ports] - average_returns[, 1] #The difference portfolio
    if (sum(is.na(average_sort_values)) == length(average_sort_values) |
        sum(is.na(average_returns)) == length(average_returns)) { #No need to append if the vector is empty (unavailable data)
      next
    } else {
      all_avg_sort_values <- rbind(all_avg_sort_values, average_sort_values) #Append the results
      all_avg_returns <- rbind(all_avg_returns, average_returns) #Append the results
    }
  }
  final_ouptut[1, ] <- round(c(apply(all_avg_sort_values, 2, mean, na.rm = T) , NA), 3) #Final average values of the sort variable
  for (i in 1:(no_of_ports + 1)) { #Loop through the portfolios and get the time series avg of returns, t-stat and alpha
    model_avg <- lm(na.omit(all_avg_returns[, i]) ~ 1) #Regress returns of each portfolio on the intercept (for weighted 
                                                       #portfolio there may be missing values and coeftest cannot handle them => remove)
    model_avg_adj <- coeftest(model_avg, vcov = NeweyWest(model_avg, lag = 6)) #Make Newey-West adjustment of std errors
    final_ouptut[2, i] <- round(model_avg_adj[1, 1], 3) #Store the intercept (=time series average)
    final_ouptut[3, i] <- paste0("(", round(model_avg_adj[1, 3], 3), ")") #Store the t-statistic
    #Regress the returns on the five factors
    factor_data <- na.omit(merge.xts(all_avg_returns[, i],
                                     fffactors[, 1:3],
                                     momentum,
                                     liquidity)) #Merge the data needed for the factor model (to secure the same 
                                                 #number of obs) + remove NAs since coeftest cannot handle it
    factor_model <- lm(factor_data[, 1] ~ factor_data[, 2] + factor_data[, 3] + factor_data[, 4] + factor_data[, 5] + 
                         factor_data[, 6])
    factor_model_adj <- coeftest(factor_model, vcov = NeweyWest(factor_model, lag = 6)) #Make Newey-West adjustment
    final_ouptut[4, i] <- round(factor_model_adj[1, 1], 3) #Store the intercept (alpha)
    final_ouptut[5, i] <- paste0("(", round(factor_model_adj[1, 3], 3), ")") #Store the t-statistic
  }
  return(final_ouptut)
}

Apart from the univariate sort, we also perform an independent bivariate portfolio analysis. In this way we can examine the relation between one sorting variable and the outcome variable after controling for some other variable. In our case we consider a sort on Size and Beta and then on Size and Skewness. The procedure is quite similar to the univariate sort.

Firstly, we divide both sorting variables into three groups based on the 30th and 70th percentile. We then form portfolios for all possible combinations resulting in 25 portfolios in total (we also include the average and difference portfolios). As in the previous case, we consider closed intervals to prevent some portfolios to be empty. Since we are concerned with an independent sorting procedure, we find the breakpoints for each variable independently and then find the stocks belonging to each portfolio by combining the breakpoints. Then as in the univariate sort, we calculate the cross-sectional averages for each portfolio in each period. Subsequently, we calculate the average and difference portfolios and lastly the difference in difference portfolio. Following Bali *et al.* (2016), we do not present the results for the average of averages portfolio (therefore, the total number of portfolios is 24).

Once we have the time series of excess returns for each portfolio, the procedure is identical to the univariate sort. The only exception is that we do not report the average values of the sorting variables.

In [10]:
#Defining a function to perform a bivariate sort (independent and weighted)
bivariate_sort <- function(sort_variable1, sort_variable2, percs1 = c(0.3, 0.7), 
                           percs2 = c(0.3, 0.7)) { #Accepts two sort variables data and required percentiles for the portfolios
  no_of_ports1 <- length(percs1) + 1 #Storing the number of portfolios of the first sort variable
  no_of_ports2 <- length(percs2) + 1 #Storing the number of portfolios of the second sort variable
  sort_variable1_name <- gsub("MDLZ.", "", colnames(sort_variable1)[1]) #Extracting the name of the first sorting variable
  sort_variable2_name <- gsub("MDLZ.", "", colnames(sort_variable2)[1]) #Extracting the name of the second sorting variable
  final_ouptut <- as.data.frame(matrix(ncol = no_of_ports1 + 2, nrow = 4*(no_of_ports2 + 2))) #Data frame for the final output
  colnames(final_ouptut) <- c(paste0(sort_variable1_name, " ", 1:no_of_ports1),
                              paste(sort_variable1_name, "Avg"),
                              paste0(sort_variable1_name, " ", paste0(no_of_ports1, "-", 1))) #Column names for clarity
  sort_variable2_ports <- c(paste0(sort_variable2_name, " ", 1:no_of_ports2), #Portfolios of the second sort variable 
                            paste(sort_variable2_name, "Avg"),                #stored to use for row names of the final output
                            paste0(sort_variable2_name, " ", paste0(no_of_ports2, "-", 1))) 
  statistics <- c("Mean", "t-statistic", "Alpha", "t-statistic (alpha)") #Statistics to paste for row names
  row.names(final_ouptut) <- as.vector(t(outer(sort_variable2_ports, statistics, FUN = "paste", sep = " - "))) #Row names for clarity
  all_avg_returns <- list() #Empty vector to which we will append the matrices of results from each period (returns)
  find_indics <- function(xts_object, percs) { #A function that returns boolean vectors for given quantiles of given data
    breakpoints <- quantile(xts_object, probs = percs, na.rm = T) #Find the breakpoints
    no_of_quants <- length(breakpoints) + 1 #Store the number of quantiles
    indics <- vector("list", no_of_quants) #Empty list to store the final indices
    for (i in 1:no_of_quants) { #Loop through the quantiles
      if (i == 1) { #For the first quantile we have only one condition
        indic <- xts_object <= breakpoints[i] #Find the stocks in the first quantile
      } else if (i == no_of_quants) { #For the last quantile we have only one condition as well
        indic <- xts_object >= breakpoints[i -  1]
      } else { #The rest of the quantiles (two conditions)
        indic <- xts_object >= breakpoints[i - 1] & xts_object <= breakpoints[i] #"=" at both inequalities to prevent empty quantiles (see book)
      }
      indics[[i]] <- indic
    }
    return(indics)
  }
  for (i in 1:nrow(sort_variable1)) { #Loop through the rows (periods) of the first sorting 
                                      #variable (we are interested in the intersection of the periods so we can loop through just one of the indices)
    current_period <- index(sort_variable1)[i] #Store the current period
    next_period <- as.Date(as.yearmon(current_period %m+% months(1)), frac = 1) #Store the next period (for the 1-ahead returns)
    if (!next_period %in% index(monthly_returns) |
        !current_period %in% index(sort_variable2)) { #In case we do not have 1-ahead returns or the values for the second variable, skip the period
      next
    }
    average_returns <- matrix(nrow = no_of_ports2 + 2,
                              ncol = no_of_ports1 + 2) #Place holder for the cross-sectional average values of the 1-ahead returns
    indics1 <- find_indics(sort_variable1[current_period],
                           percs1) #Divide the first sorting variable into quantiles (get the boolean values for the stocks)
    indics2 <- find_indics(sort_variable2[current_period],
                           percs2) #Divide the second sorting variable into quantiles (get the boolean values for the stocks)
    indics_comb <- expand.grid(indics1, indics2) #Make all possible combinations for all quantiles of both variables
    indics_for_mat <- expand.grid(1:no_of_ports1, 1:no_of_ports2) #Indices for writing the returns in the matrix for each period
    for (j in 1:nrow(indics_comb)) { #Loop through the portfolios
      final_indics <- which(unlist(indics_comb[j, 1]) & unlist(indics_comb[j, 2])) #Find the column indices for a given portfolio
      if (length(final_indics) == 0) { #If there are no stocks in the portfolio, skip
        next
      }
      mc_weights <- as.numeric(market_cap[current_period, final_indics]) #Storing the market cap weights
      if (length(mc_weights) == 0) { #If we do not have the market cap data, we have to skip
        next
      }
      mc_weights[is.na(mc_weights)] <- 0 #0 value for missings = they have no weight
      average_returns[indics_for_mat[j, 2], indics_for_mat[j, 1]] <- weighted.mean(as.numeric(monthly_returns[next_period, final_indics]),
                                                                                   w = mc_weights, na.rm = T)  
    }
    for (j in 1:(ncol(average_returns) - 2)) { #Calculate the average portfolio for the second sorting variable
      average_returns[nrow(average_returns) - 1, j] <- mean(average_returns[1:(nrow(average_returns) - 2), j], na.rm = T)
    }
    for (j in 1:(nrow(average_returns) - 2)) { #Calculate the average portfolio for the first sorting variable
      average_returns[j, ncol(average_returns) - 1] <- mean(average_returns[j, 1:(ncol(average_returns)-2)], na.rm = T)
    }
    for (j in 1:(ncol(average_returns)- 1)) { #Calculate the difference portfolios of the second sorting variable (last row)
      average_returns[nrow(average_returns), j] <- average_returns[nrow(average_returns) - 2, j] - average_returns[1, j] 
    }
    for (j in 1:nrow(average_returns)) { #Calculate the difference portfolios of the first sorting variable + the diff in diff portfolio (last column)
      average_returns[j, ncol(average_returns)] <- average_returns[j, ncol(average_returns) - 2] - average_returns[j, 1]
    }
    all_avg_returns[[length(all_avg_returns) + 1]] <- average_returns #Append the matrix with results for the current period
    names(all_avg_returns)[length(all_avg_returns)] <- as.character(current_period) #Name the element in the list with the period for later usage
  }
  indics_for_ports <- expand.grid(1:(no_of_ports1 + 2), 1:(no_of_ports2 + 2)) #Store the indices for each portfolio in the returns matrix for the loop
  final_returns <- rep(NA, ((no_of_ports1 + 2)*(no_of_ports2 + 2))) #A vector to store the final returns for each portfolio
  final_t_stats <- rep(NA, ((no_of_ports1 + 2)*(no_of_ports2 + 2))) #A vector to store the final t-statistics for each portfolio
  final_alphas <- rep(NA, ((no_of_ports1 + 2)*(no_of_ports2 + 2))) #A vector to store the final alphas for each portfolio
  final_t_stats_alphas <- rep(NA, ((no_of_ports1 + 2)*(no_of_ports2 + 2))) #A vector to store the final t-statistics for alphas for each portfolio
  for (i in 1:((no_of_ports1 + 2)*(no_of_ports2 + 2))) { #Loop through the portfolios and get the time series avg of returns, t-stat and alpha
    if (indics_for_ports[i, 1] == (no_of_ports1 + 1) & indics_for_ports[i, 2] == (no_of_ports2 + 1)) { #Skip the average of averages portfolio
      next
    }
    data_to_use <- xts(unlist(lapply(all_avg_returns, "[", indics_for_ports[i, 2], indics_for_ports[i, 1])),
                       order.by = as.Date(names(all_avg_returns))) #Get the time series for each portfolio
    model_avg <- lm(na.omit(data_to_use) ~ 1) #Regress returns of each portfolio on the intercept (for weighted 
                                              #portfolio there may be missing values and coeftest cannot handle them => remove)
    model_avg_adj <- coeftest(model_avg, vcov = NeweyWest(model_avg, lag = 6)) #Make Newey-West adjustment of std errors
    final_returns[i] <- model_avg_adj[1, 1] #Store the intercept (=time series average)
    final_t_stats[i] <- model_avg_adj[1, 3] #Store the t-statistic
    #Regress the returns on the five factors
    factor_data <- na.omit(merge.xts(data_to_use,      #Merge the data needed for the factor 
                                    fffactors[, 1:3],  #model (to secure the same number of obs) + remove NAs since coeftest cannot handle it 
                                    momentum,
                                    liquidity)) 
    factor_model <- lm(factor_data[, 1] ~ factor_data[, 2] + factor_data[, 3] + factor_data[, 4] + factor_data[, 5] + 
                         factor_data[, 6])
    factor_model_adj <- coeftest(factor_model, vcov = NeweyWest(factor_model, lag = 6)) #Make Newey-West adjustment
    final_alphas[i] <- factor_model_adj[1, 1] #Store the intercept (alpha)
    final_t_stats_alphas[i] <- factor_model_adj[1, 3] #Store the t-statistic
  }
  final_result <- list(final_returns, final_t_stats, final_alphas, final_t_stats_alphas)
  for (i in 1:4) { #Loop through the list of results
    results_mat <- matrix(final_result[[i]], ncol = no_of_ports1 + 2, byrow = T) #Reshape the vector to matrix to facilitate filling in the final output
    if (i %in% c(2, 4)) { #Wrap the t-statistics in brackets for nicer output
      final_ouptut[seq(i, nrow(final_ouptut), 4), ] <- paste0("(", round(results_mat, 3), ")")  
    } else {
      final_ouptut[seq(i, nrow(final_ouptut), 4), ] <- round(results_mat, 3) #Fill the appropriate rows with results for each statistic
    }
  }
  return(final_ouptut)
}

The last part of our analysis entails performing the Fama-Macbeth regression. In this method we make the assumption that the relationship between the variables in question is linear. We firstly consider the sole relationship between Skewness and excess returns and then we also include the variables Size and Beta into the regression.

The procedure is somehow similar to the portfolio analysis in the way that we first estimate individual cross-sectional regressions for each period, we store the coefficients, and then we calculate the average over all periods. In this case we present only the robust *t*-statistics along with the average Adjusted R-squared and average number of observations used.

In [11]:
#Defining a function to perform the Fama-Macbeth regression
fama_macbeth <- function(indep_var) { #Expects a list of xts objects with independent variables
  cross_sec_coefs <- c() #Empty vector to which we will append the coefficients for each period
  adj_r_squared <- c() #Empty vector for adjusted R-squared in each period
  no_of_obs <- c() #Empty vector to store the number of observations in each period
  for (i in 1:nrow(monthly_returns)) { #Loop through the periods
    next_period <- index(monthly_returns)[i] #Store the period from which the one ahead returns come
    current_period <- as.Date(as.yearmon(next_period %m-% months(1)), frac = 1) #Store the period for the independent variables
    move_to_next_period <- F #Setting up an indicator for incomplete periods
    for (j in 1:length(indep_var)) { #Check if the current period is contained in all independent variables
      if (current_period %in% index(indep_var[[j]])) { #If it is, move to next variable
        next
      } else { #If it is not, set the indicator and break
        move_to_next_period <- T
        break
      }
    }
    if (move_to_next_period == T) { #If one of the independent variables does not contain the required period, move to next period
      next
    }
    cross_sec_data <- as.data.frame(matrix(ncol = length(indep_var) + 1, nrow = ncol(monthly_returns))) #Data holder for each period
    colnames(cross_sec_data) <- c("Returns", names(indep_var)) #Column names that will be used in the formula
    cross_sec_data[, 1] <- as.numeric(monthly_returns[next_period]) #Fill in the data for the dependent variable
    for (j in 1:length(indep_var)) { #Fill in the data for each independent variable
      cross_sec_data[, j + 1] <- as.numeric(indep_var[[j]][current_period])
    }
    form <- paste("Returns", paste(colnames(cross_sec_data)[2:ncol(cross_sec_data)], collapse = "+"), sep = "~") #Define the formula
    cross_sec_model <- lm(form, data = cross_sec_data) #Estimate the model
    cross_sec_coefs <- rbind(cross_sec_coefs, cross_sec_model$coefficients) #Append the coefficients
    adj_r_squared[length(adj_r_squared) + 1] <- summary(cross_sec_model)$adj.r.squared #Append the adjusted r_squared
    no_of_obs[length(no_of_obs) + 1] <- nrow(na.omit(cross_sec_data)) #Append the number of observations
  }
  final_output <- as.data.frame(matrix(nrow = 2,                           #Place holder to store the final 
                                       ncol = ncol(cross_sec_coefs) + 2))  #output (time series means and t-statistics + adjusted r squared and no of obs)
  colnames(final_output) <- c(colnames(cross_sec_coefs), "Adj. R-squared", "n") #Rename the columns for clarity
  row.names(final_output) <- c("Mean", "t-statistic") #Rename the rows for clarity
  for (i in 1:ncol(cross_sec_coefs)) { #Calculate the time series mean and t-statistics for each coefficient
    time_model <- lm(cross_sec_coefs[, i] ~ 1) #Regress each time series on a constant to get the mean estimate
    time_model_adj <- coeftest(time_model, vcov = NeweyWest(time_model, lag = 4)) #Newey-West adjustment of the errors
    final_output[1, i] <- round(time_model_adj[1, 1], 3) #Store the mean
    final_output[2, i] <- paste0("(", round(time_model_adj[1, 3], 3), ")") #Store the t-statistic
  }
  final_output[1, ncol(cross_sec_coefs) + 1] <- round(mean(adj_r_squared, na.rm = T), 3) #Calculate the average adjusted R-squared
  final_output[1, ncol(cross_sec_coefs) + 2] <- round(mean(no_of_obs, na.rm = T), 3) #Calculate average number of observations
  return(final_output)
}

## 4. Results
<a id='results'></a> 

In this section we present the results of our analyses. We start with the univariate sort.

### Univariate sort
<a id='univariate'></a> 

The first sorting variable of interest is Beta. It measures the stock's sensitivity to market. The table below presents the results of the univariate sort with equally-weighted portfolios. The average values of the sorting variable are strictly increasing for the portfolios which suggests that the stocks are divided correctly. As for the results of the analysis, we failed to find any evidence of a statistically significant relationship between Beta and one-ahead returns since the average value for the difference portfolio is statistically indistinguishable from zero (t-statistic 1.59 and 1.35 for the excess returns and alpha, respectively). This result is in line with the existing literature. Nevertheless, the individual average values as well as the alphas are statistically significant which indicates that each portfolio generates positive returns on average.

In [12]:
#Univariate sort on betas (equally-weighted)
univariate_sort_betas <- univariate_sort(betas) #Equally weighted
univariate_sort_betas

1       2       3       4       5       5-1    
Mean of Beta              0.532   0.815   1.003   1.201   1.56    NA     
Mean of one-ahead returns 0.011   0.012   0.014   0.019   0.025   0.014  
t-statistic               (2.863) (2.681) (2.267) (2.741) (2.775) (1.586)
Alpha                     0.016   0.014   0.016   0.021   0.028   0.012  
t-statistic (alpha)       (3.754) (3.182) (3.029) (3.134) (3.019) (1.354)

In the previous case we have considered equally-weighted portfolios. It is a common practice to also present value-weighted portfolios (some studies show exclusively value-weighted portfolios). The results are available below. We can see that our conclusions remain unchanged, there is not a significant relationship ($\alpha$ of the difference portfolio has a t-statistic of -0.03). Even though that the CAPM predicts a positive and significant relationship between Beta and returns, empirical research does not seem to find corresponding evidence.

In [13]:
#Univariate sort on betas (value-weighted)
univariate_sort_weighted_betas <- univariate_sort(betas, weighted = T) #Market cap weighted
univariate_sort_weighted_betas

1       2       3       4       5       5-1     
Mean of Beta              0.525   0.811   1.003   1.202   1.562   NA      
Mean of one-ahead returns 0.015   0.016   0.018   0.022   0.02    0.005   
t-statistic               (3.996) (4.072) (3.061) (3.813) (2.714) (0.665) 
Alpha                     0.021   0.019   0.019   0.021   0.02    0       
t-statistic (alpha)       (5.548) (3.868) (3.531) (3.342) (2.33)  (-0.028)

The next sorting variable we consider is Size. Once again, we can see that the average values for the sorting variable are monotonic which supports the correctness of the procedure. As opposed to Beta, we find a very statistically significant relationship with the one-ahead returns. The average value for the difference portfolio suggests that small firms from the first portfolio generate on average returns higher by 1.6% than the large stocks in the last portfolio (mean of returns and $\alpha$ have the same value). The average values are in fact monotonically decreasing across the portfolios (when considering one-ahead returns). However, we can observe a distinct drop in returns between the first and second portfolio. This suggests that the relationship is likely driven by a few very small stocks. The relationship is robust even after controlling for the 3 Fama-French factors, momentum, and liquidity. As in the previous case, this result is common in empirical research and matches the findings of Bali *et al.* (2016).

In [14]:
#Univariate sort on size (equally weighted)
univariate_sort_size <- univariate_sort(size)
univariate_sort_size

1       2       3       4       5       5-1     
Mean of Size              1.604   2.299   2.827   3.439   4.711   NA      
Mean of one-ahead returns 0.03    0.018   0.016   0.014   0.014   -0.016  
t-statistic               (5.285) (3.666) (3.666) (3.551) (3.724) (-6.151)
Alpha                     0.033   0.019   0.019   0.016   0.017   -0.016  
t-statistic (alpha)       (4.66)  (3.378) (3.277) (3.152) (3.748) (-5.109)

Moreover, the findings hold even for value-weighted portfolios. Both the average one-ahead returns as well as the alpha of the difference portfolio are statistically significant and negative. In addtion, they differ only slightly in magnitude (-1.3% and -1.4%, respectively).

In [15]:
#Univariate sort on size (value-weighted)
univariate_sort_weighted_size <- univariate_sort(size, weighted = T) #Market cap weighted
univariate_sort_weighted_size

1       2       3       4       5       5-1     
Mean of Size              1.604   2.299   2.827   3.439   4.711   NA      
Mean of one-ahead returns 0.028   0.017   0.016   0.014   0.015   -0.013  
t-statistic               (5.254) (3.592) (3.665) (3.568) (3.79)  (-4.51) 
Alpha                     0.031   0.019   0.019   0.016   0.017   -0.014  
t-statistic (alpha)       (4.58)  (3.315) (3.24)  (3.248) (3.943) (-3.706)

In the last part of the univariate portfolio analysis, we consider a sorting variable that captures the skewness of the returns. Even though the individual average values are significant, there does not appear to be a statistically significant relationship between our proxy for Skewness and one-ahead returns. The average values are in fact very similar across portfolios leaving no evidence of an observable pattern. As discussed above, this is likely due to the nature of the utilized proxy since Bali *et al.* (2016) also failed to find a significant relationship when considering 12 months of daily data for the calculation of total skewness. A monthly measure of total skewness appears to be more appropriate since it is less noisy.

In [16]:
#Univariate sort on skewness (equally weighted)
univariate_sort_skewness <- univariate_sort(skewness_data)
univariate_sort_skewness

1       2       3       4       5       5-1    
Mean of Skewness          -1.109  -0.29   -0.035  0.248   1.21    NA     
Mean of one-ahead returns 0.015   0.017   0.015   0.017   0.018   0.003  
t-statistic               (3.104) (3.128) (2.55)  (2.598) (2.981) (0.964)
Alpha                     0.019   0.019   0.017   0.019   0.02    0.001  
t-statistic (alpha)       (3.859) (4.344) (3.029) (3.071) (3.538) (0.471)

The value-weighted portfolios yield similar results. The relationship appears to be insignificant.

In [17]:
#Univariate sort on skewness (value-weighted)
univariate_sort_weighted_skewness <- univariate_sort(skewness_data, weighted = T) #Market cap weighted (rbind warning should be okay, just NAs)
univariate_sort_weighted_skewness

1       2       3       4       5       5-1     
Mean of Skewness          -1.112  -0.305  -0.049  0.235   1.204   NA      
Mean of one-ahead returns 0.018   0.019   0.016   0.017   0.016   -0.001  
t-statistic               (4.346) (3.716) (3.126) (3.481) (3.699) (-0.386)
Alpha                     0.021   0.022   0.017   0.021   0.018   -0.004  
t-statistic (alpha)       (4.196) (3.992) (2.97)  (3.399) (4.052) (-0.882)

### Bivariate sort
<a id='bivariate'></a> 

In the second part of our analysis we focus on bivariate portfolio analysis. This allows us to investigate the effect of a sorting variable on the outcome variable while controlling for a second sorting variable. Firstly, we consider Size and Beta. Since the sort is independent, we can inspect the effect of both variables. The table below presents the results using value-weighted portfolios. Our main aim is to investigate the difference portfolios of one variable at different levels of the other variable. We can see that for low Beta stocks, the Size difference portfolio generates statistically significant negative returns after adjusting for risk (-0.7% with t-statistic of -2.58). For the second group of Betas the alpha is only marginally statistically significant (t-statistic of -1.83). For the high Beta stocks, the alpha is highly statistically significant again (-1.5% with t-statistic -3.67). For the average values of $\beta$, the difference portfolio generates statistically significant results as well (t-statistic of -3.97 for alpha). As for the average Size portfolio, the results do not deviate across different levels of Beta. Therefore, the relationship does not appear to differ accross different levels of Beta.

On the other hand, the Beta difference portfolio is insignificant accross all levels of Size if we consider the risk-adjusted returns (only one-ahead returns for the lowest Size group are statistically significant but not robust to risk adjustment). In addition, the difference in difference portfolio is insignificant as well. In fact, the unadjusted returns are marginally statistically significant but the relationship is not robust to the risk-adjustment. This suggests that we find little to no evidence that the relationship between Size and returns is different for different values of Beta and vice versa. Based on our analysis, the significant relationship between Size and returns cannot be explained by different values of Beta.

If we were to compare our findings to Bali *et al.* (2016), we arrive to slightly different results. Firstly, the researchers do not find a significant risk-adjusted return of the Size difference portfolio for average values of Beta (the study in fact utilizes Market Cap but given the non-parametric nature of the analysis, this should not make a large difference). In addition, they discover the strongest effect of Size for the lowest values of Beta which is not our case. As already discussed, this may be due to the fact that our data set fails to capture stocks with low or negative sensitivity to the market. This is also projected in the results for Beta since the authors find a significant negative abnormal return for the Beta difference portfolio for average values of Size.

In [18]:
#Bivariate sort on size and beta
bivariate_sort_size_beta <- bivariate_sort(size, betas)
bivariate_sort_size_beta

Size 1  Size 2  Size 3   Size Avg Size 3-1
Beta 1 - Mean                  0.021   0.015   0.015    0.017    -0.006  
Beta 1 - t-statistic           (5.132) (4.418) (4.603)  (5.135)  (-1.719)
Beta 1 - Alpha                 0.027   0.02    0.02     0.022    -0.007  
Beta 1 - t-statistic (alpha)   (5.594) (4.933) (5.397)  (5.688)  (-2.582)
Beta 2 - Mean                  0.023   0.018   0.018    0.02     -0.005  
Beta 2 - t-statistic           (3.486) (2.815) (3.342)  (3.282)  (-1.962)
Beta 2 - Alpha                 0.024   0.019   0.019    0.021    -0.005  
Beta 2 - t-statistic (alpha)   (3.496) (2.937) (3.572)  (3.372)  (-1.826)
Beta 3 - Mean                  0.034   0.022   0.019    0.026    -0.016  
Beta 3 - t-statistic           (4.186) (2.841) (3.129)  (3.57)   (-3.49) 
Beta 3 - Alpha                 0.034   0.025   0.019    0.026    -0.015  
Beta 3 - t-statistic (alpha)   (3.912) (2.751) (2.529)  (3.222)  (-3.67) 
Beta Avg - Mean                0.026   0.018   0.017    NA       -0.009  
Beta Avg - t-statistic         (4.334) (3.294) (4.025)  (NA)     (-3.544)
Beta Avg - Alpha               0.028   0.021   0.019    NA       -0.009  
Beta Avg - t-statistic (alpha) (4.352) (3.364) (3.812)  (NA)     (-3.973)
Beta 3-1 - Mean                0.013   0.007   0.003    0.009    -0.01   
Beta 3-1 - t-statistic         (2.242) (1.032) (0.565)  (1.603)  (-1.921)
Beta 3-1 - Alpha               0.007   0.005   -0.001   0.004    -0.008  
Beta 3-1 - t-statistic (alpha) (1.119) (0.697) (-0.209) (0.623)  (-1.557)

In the second part of the bivariate portfolio analysis we consider Size and Skewnes. The Size difference portfolio appears to generate statistically significant and negative excess one-ahead returns and alphas across all levels of Skewness. On the other hand, for the Skewness difference portfolio the unadjusted returns are significant only for the low values of Size (t-statistics of 2.12). In addition, the significance dissipates after addjusting for the 5 factors (t-statistics of 1.03). Moreover, as in the previous case, the insignificance of the returns of the difference in difference portfolio indicates that we find no evidence that the relationship between Size and returns differs based on various levels of Skewness. The same applies vice versa.

In [19]:
#Bivariate sort on size and skewness
bivariate_sort_size_skewness <- bivariate_sort(size, skewness_data)
bivariate_sort_size_skewness

Size 1  Size 2   Size 3   Size Avg Size 3-1
Skewness 1 - Mean                  0.023   0.018    0.016    0.019    -0.007  
Skewness 1 - t-statistic           (4.166) (4.087)  (3.742)  (4.164)  (-2.525)
Skewness 1 - Alpha                 0.028   0.02     0.019    0.022    -0.008  
Skewness 1 - t-statistic (alpha)   (4.172) (4.192)  (4.041)  (4.29)   (-2.885)
Skewness 2 - Mean                  0.027   0.018    0.018    0.021    -0.009  
Skewness 2 - t-statistic           (4.021) (3.026)  (3.344)  (3.59)   (-2.881)
Skewness 2 - Alpha                 0.028   0.022    0.02     0.023    -0.008  
Skewness 2 - t-statistic (alpha)   (4.02)  (2.993)  (3.541)  (3.617)  (-2.642)
Skewness 3 - Mean                  0.032   0.018    0.016    0.022    -0.016  
Skewness 3 - t-statistic           (4.289) (2.922)  (3.937)  (3.971)  (-2.825)
Skewness 3 - Alpha                 0.031   0.021    0.017    0.023    -0.014  
Skewness 3 - t-statistic (alpha)   (4.444) (2.977)  (4.106)  (4.08)   (-2.929)
Skewness Avg - Mean                0.027   0.018    0.016    NA       -0.011  
Skewness Avg - t-statistic         (4.269) (3.349)  (3.999)  (NA)     (-3.458)
Skewness Avg - Alpha               0.029   0.021    0.019    NA       -0.01   
Skewness Avg - t-statistic (alpha) (4.332) (3.396)  (4.126)  (NA)     (-3.687)
Skewness 3-1 - Mean                0.008   0        0        0.003    -0.008  
Skewness 3-1 - t-statistic         (2.122) (-0.006) (-0.057) (1.171)  (-1.705)
Skewness 3-1 - Alpha               0.003   0.001    -0.003   0        -0.006  
Skewness 3-1 - t-statistic (alpha) (1.026) (0.217)  (-0.749) (0.231)  (-1.208)

### Fama-MacBeth regression
<a id='fm_regression'></a> 

In the last part of this section we present the results of the Fama-MacBeth regression. Firstly, we consider the sole relationship between Skewness and one-ahead excess returns. As the table below indicates, there does not seem to be any significant relationship between the variables. This is not surprising since we did not find any relationship in the previous non-parametric analyses and in the case of Fama-MacBeth regression we impose restrictions on the linearity of the relationship. The average value of Adjusted R-squared indicates that Skewness explains virtually no variation in the dependent variable. The intercept is positive and statistically significant which would suggest that the stocks in our portfolio generate on average positive excess returns after controlling for Skewness. Even though the Skewness coefficient is not statistically different from zero, we can for the sake of the analyses take a look at its economic implications. An increase of one standard deviation in Skewness (0.97) results in an increase in excess returns of 0.19% (to avoid confusion, the increases is 0.00194 in returns since in our analysis they are measured in decimals, therefore 0.19% if we scale them to percents).

In [20]:
#Fama-Macbeth for skewness
FM_skewness <- fama_macbeth(list(Skewness = skewness_data))
FM_skewness

(Intercept) Skewness Adj. R-squared n      
Mean        0.016       0.002    0.006          248.446
t-statistic (3.142)     (1.047)     NA               NA

In [21]:
#Economic magnitude of the Skewness coefficient
as.numeric(FM_skewness[1, 2]) * sum_stats[2, 2] * 100

[1] 0.194

Lastly, we inspect the relationship between all previously considered sorting variables and the one-ahead excess returns. The results in the table below show that only Size has a statistically significant effect on the dependent variable (The effect of Beta is marginally statistically significant). The relationship is negative which further supports our previous findings and matches the extant literature. The Market Capitalization of a firm appears to be a very important determinant of its future returns. Nevertheless, the average Adjusted R-squared shows that we still fail to capture majority of the variation in the dependent variable. As in the previous case, the intercept is statistically significant and positive suggesting abnormal returns even after controlling for the mentioned factors. As for the economic magnitude, our model predicts that a one standard deviation increase in Size (1.12) results in a decrease in one-ahead excess returns of 0.45%. Moreover, the difference between the 95th and 5th percentiles is -1.44% which indicates that very large stocks generate monthly excess returns that are lower by 1.44% than the returns of very small stocks.

In [22]:
#Fama-Macbeth for the three sort variables
indep_var <- list(Skewness = skewness_data, Size = size, Beta = betas)
FM_three_sorts <- fama_macbeth(indep_var)
FM_three_sorts

(Intercept) Skewness Size     Beta    Adj. R-squared n      
Mean        0.017       -0.001   -0.004   0.013   0.099          244.045
t-statistic (3.608)     (-0.73)  (-4.158) (1.795)    NA               NA

In [23]:
#Economic magnitude of the Size coefficient
as.numeric(FM_three_sorts[1, 3]) * sum_stats[4, 2] * 100 #One standard deviation
as.numeric(FM_three_sorts[1, 3]) * (sum_stats[4, 10] - sum_stats[4, 6]) * 100 #Difference between 95% and 5% percentile

[1] -0.448

[1] -1.444

## 5. Conclusion
<a id='conclusion'></a> 

To conclude, we derived methods for univariate and bivariate sorts and Fama-MacBeth regression and employed these on a sample of 250 stocks' one-ahead monthly excess returns, utilizing Skewness, Size, and Betas as sorting variables. The data we used range from January 2007 to February 2022.

Our findings are mostly in line with the literature. As for the univariate sort, we created 5 equally-weighted and 5 value-weighted portfolios for the three sorting variables. Additionally, we derived difference “5-1“ portfolios in each case. For each portfolio, we report the average value of sorting variable, the time-series averages with corresponding Newey-West t-statistics of one-month-ahead excess return and Fama-French-Carhart-Pastor-Stambaugh alpha (with its t-statistics). 

When sorted by Beta, we found no statistically significant relationship between the Beta and one-ahead excess returns, as the value for the difference portfolio was not statistically different from zero. This holds both for equally-weighted and value-weighted portfolios.
As for the Size-based sort, there appears to be a statistically significant relationship with excess returns, indicating that small firms (those from the first quintile) generate on average returns higher by 1.6% than larger firms (form the fifth quintile). This finding holds for both equally and value-weighted portfolios (in this case returns are higher by 1.3%), and for a mean of returns, as well as for alpha values. Portfolios sorted by Skewness do not seem to have the excess returns related to the sorting variable, in any case, the relationships are statistically insignificant.
All our findings from the univariate sorting of portfolios go in line with the findings of Bali *et al.* (2016).

When sorting the portfolios by two variables, we used two breakpoints on the 30th and 70th percentiles, resulting in 5*5 portfolios in total (3 portfolios divided by breakpoints, one "3-1" difference portfolio, and one average portfolio, therofe 5 portfolios for each sorting variable). Similarly as before, we reported the average value of sorting variable, the time-series averages with corresponding Newey-West t-statistics of one-month-ahead excess return and Fama-French-Carhart-Pastor-Stambaugh alpha (with its t-statistics). 

Firstly we sorted by Size and Beta. The effect of Size on returns seems to be consistently negative (Size difference portfolio has negative mean and alpha values) and significant across various levels of Beta. Contrary, the Beta difference portfolio is insignificant across all levels of Size and the difference-in-difference portfolio is insignificant as well. Therefore, we conclude that the significant relationship between Size and returns cannot be explained by different values of Beta. Results of Bali *et al.* (2016) differ slightly, however, in general the results are similar. The difference might be caused by our dataset not having stocks with negative Betas. Secondly, we sorted by Size and Skewness. Similarly as in the previous case, we concluded that the relationship between Size and returns is constant for various levels of Skewness (and vice versa), as the effect of the difference-in-difference portfolio was insignificant.

Lastly, we performed the Fama-MacBeth regression analysis. When considering solely the relationship between Skewness and one-ahead excess returns, the estimated effect is insignificant, and thus inconclusive. The estimated intercept is significantly positive, suggesting that the stocks in our portfolio generate on average positive excess returns after controlling for Skewness. When Size and Beta are added into the regression as explanatory variables, we can find Size has a statistically negative effect (other estimates are insignificant). This follows our previous findings, as well as the results of Bali *et al.* (2016). 

## Appendix
<a id='appendix'></a> 

### Appendix A

#### Data preparation
<a id='appendix_a'></a> 

We performed the following steps to obtain the final data utilized in our analyses. Firstly, we loaded the data on Adjusted Close prices and Market capitalization which we collected in Homework 1. In addition, we downloaded the Fama-French 3 factors as well as the Momentum factor from [here](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). The last required piece of data was Liquidity which we accessed at the [website](https://faculty.chicagobooth.edu/lubos-pastor/data) of Ľuboš Pástor.

In [ ]:
#Loading the data
yahoo_data <- readRDS("Raw data/list_yahoo.RData") #Adjusted close prices and volumes
market_cap <- readRDS("Raw data/market_cap_data.RData") #Market capitalization
fffactors_monthly <- read_csv("Raw data/F-F_Research_Data_Factors.CSV", skip = 2, n_max = 1149) #Need to limit the number of rows 
                                                                                                #since the file contains yearly data as well
fffactors_daily <- read_csv("Raw data/F-F_Research_Data_Factors_daily.CSV", skip = 3, n_max = 25210) #Last row is some copyright => skip
momentum <- read_csv("Raw data/F-F_Momentum_Factor.CSV", skip = 12, n_max  = 1144, na = c("", "NA", -99.99, -999))
liquidity <- read.delim("Raw data/liq_data_1962_2021.txt", skip = 10, na = c("", "NA", -99.99, -999, -99))

Next, we define an auxilliary function which removes empty lines from an xts object (for future use).

In [51]:
#Defining a function to strip an xts object of lines with 0 observations (all NAs)
strip_empty_lines <- function(xts_object) {
  na_sums <- apply(xts_object, 1, function(x) {sum(is.na(x))})
  indic <- na_sums < ncol(xts_object)
  return(xts_object[indic])
}

Then, we extract the Adjusted Close price.

In [52]:
#Extracting the Adjusted Close Price
daily_adj_close <- lapply(yahoo_data, "[", , 6) #The unused argument is not an error, we have to skip it to specify columns

In order to calculate the excess returns, we need the market risk premium. Therefore, we extract it from the Fama-French factors data set. We also convert the daily observations to monthly and save the resulting data for future use in the portfolio analysis.

In [53]:
#Extracting market risk premium and the risk-free rate
fffactors_monthly <- as.data.frame(fffactors_monthly) #Converting from tibble to a data frame
fffactors_daily <- as.data.frame(fffactors_daily) #Converting from tibble to a data frame
colnames(fffactors_daily)[1] <- "Date"
colnames(fffactors_monthly)[1] <- "Date"
fffactors_daily$Date <- as.Date(as.character(fffactors_daily$Date), "%Y%m%d") #Creating a date (daily)
fffactors_monthly$Date <-  as.Date(paste0(as.character(fffactors_monthly$Date),"01"), "%Y%m%d") + months(1) - days(1) #Creating a date (end of month)
fffactors_monthly[, 2:5] <- fffactors_monthly[, 2:5] / 100 #Scale by 100 since the data is in percents
fffactors_daily[, 2:5] <- fffactors_daily[, 2:5] / 100 #Scale by 100 since the data is in percents
market_prem_daily <- xts(fffactors_daily$`Mkt-RF`, order.by = fffactors_daily$Date)
names(market_prem_daily) <- "Mkt_RF_daily"
market_prem_monthly <- xts(fffactors_monthly$`Mkt-RF`, order.by = fffactors_monthly$Date)
names(market_prem_monthly) <- "Mkt_RF_monthly"
rf_daily <- xts(fffactors_daily$RF, order.by = fffactors_daily$Date)
names(rf_daily) <- "RF_daily"
rf_monthly <- xts(fffactors_monthly$RF, order.by = fffactors_monthly$Date)
names(rf_monthly) <- "RF_monthly"
fffactors_monthly_xts <- as.xts(fffactors_monthly[, 2:5], order.by = fffactors_monthly$Date) #Converting monthly factors to xts
fffactors_monthly_xts <- fffactors_monthly_xts["2007/"] #Restricting the period since we do not need older data
#saveRDS(fffactors_monthly_xts, file = "Final data/fffactors_monthly.RData") #Saving for future use

Now we can calculate the daily and monthly excess returns. The former will be used for the calculation of various measures such as beta and skewness. We save the latter for the portfolio analysis

In [54]:
#Calculating excess returns
calc_excess_returns <- function(xts_object, daily = T) { #Defining a function to calculate excess returns
  if (daily == T) { #For daily returns
    result <- dailyReturn(xts_object)
    result <- result[2:nrow(result)] #Disregard the first observation (dailyReturn sets it to 0)
    risk_free_rate <- rf_daily[index(result)] #Define daily risk free rate
  } else { #For monthly returns
    result <- monthlyReturn(xts_object)
    risk_free_rate <- rf_monthly[index(result)]
  }
  return(result - risk_free_rate) #Substract risk free rate from the returns
}
#Daily excess returns
daily_returns <- lapply(daily_adj_close, calc_excess_returns) #Applying the function
daily_return_names <- gsub(".Adjusted", ".DailyReturns", unlist(lapply(daily_adj_close, names))) #New names
for (i in 1:length(daily_returns)) { #Renaming the columns for clarity
  names(daily_returns[[i]]) <- daily_return_names[i]
}
#Monthly excess returns
monthly_returns <- lapply(daily_adj_close, calc_excess_returns, daily = F)
monthly_return_names <- gsub(".Adjusted", ".MonthlyReturns", unlist(lapply(daily_adj_close, names))) #New names
for (i in 1:length(monthly_returns)) { #Renaming the columns for clarity
  names(monthly_returns[[i]]) <- monthly_return_names[i]
}
#Merge monthly excess returns
monthly_returns_merged <- do.call(merge.xts, monthly_returns)
#saveRDS(monthly_returns_merged, file = "Final data/monthly_excess_returns.RData") #Saving for future use

In the following step we define a function that calculates a specified monthly measure from past 12 months of daily data. In order to calculate the measure in a given period, we require at least 200 data points which is a standard for 12 months of daily data.

In [55]:
#Defining a generic function to calculate a monthly measure from past 12 months of daily data
calc_measure <- function(xts_object, func, measure_name) { #Expects a series of monthly returns, the function to calculate the required 
                                                           #measure (skewness, beta..) and the name of the measure
  result <- xts(rep(NA, nrow(xts_object)), order.by = index(xts_object)) #Empty xts object for the results
  names(result) <- gsub("MonthlyReturns", measure_name, names(xts_object)) #Naming the resulting object for clarity 
  ind <- which(monthly_return_names == names(xts_object)) #Get the rank of the stock in the list (1-250)
  for (i in 1:nrow(xts_object)) { #Loop through the rows of the object
    end_date <- index(xts_object)[i] #The last day that the daily returns will be used for computations
    start_date <- end_date %m-% months(12) #The start date is 12 months back as required
    df_to_use <- daily_returns[[ind]][paste0(start_date, "/", end_date)] #Extracting the subset we will 
                                                                         #use for the computation (12 months of daily returns)
    if (length(df_to_use) < 200){ #We require at least 200 data points, otherwise we won't calculate
      next
    } else {
      result[i] <- func(na.omit(df_to_use)) #Apply the specified function
    }
  }
  return(result)
}

We then apply the function on monthly returns for each of the 250 stocks. The first measure we calculate is Skewness (It is the third sorting variable that was assigned to our group). We save the result for future use.

In [56]:
#Calculating skewness
skewness_data <- lapply(monthly_returns, calc_measure, func = skewness, measure_name = "Skewness") #Applying the function 
                                                                                                   #(skewness from the moments package)
#Merging skewness data
skewness_data_merged <- do.call(merge.xts, skewness_data)
skewness_data_merged <- strip_empty_lines(skewness_data_merged) #Remove empty lines from data (periods with no data)
#saveRDS(skewness_data_merged, file = "Final data/skewness.RData") #Saving for future use

Another sorting variable that we utilize in the portfolio analysis is Beta. We define a function that estimates beta for a given time series and then we pass it as an argument to the function above.

In [57]:
#Calculating betas
calc_beta <- function(xts_object) { #Defining a function to calculate the beta (based on a series of daily returns)
  model_res <- lm(xts_object ~ market_prem_daily[index(xts_object)]) #Regress market premium on daily excess returns
  return(model_res$coefficients[[2]]) #Extract the estimated coefficient on market premium (beta)
}
#Applying the function
betas <- lapply(monthly_returns, calc_measure, func = calc_beta, measure_name = "Beta")
#Merging the betas
betas_merged <- do.call(merge.xts, betas)
betas_merged <- strip_empty_lines(betas_merged)
#saveRDS(betas_merged, file = "Final data/betas.RData") #Saving for future use

The last sorting variable we will use is Size. It can be calculated directly from Market Capitalization. We save the data for both of those factors.

In [58]:
#Extracting monthly market cap data
market_cap_monthly <- lapply(market_cap, to.monthly, OHLC = F, indexAt = "lastof") #Converting daily to monthly
market_cap_monthly_merged <- do.call(merge.xts, market_cap_monthly) #Merging
market_cap_monthly_merged <- strip_empty_lines(market_cap_monthly_merged)
#saveRDS(market_cap_monthly_merged, file = "Final data/market_cap_monthly.RData") #Saving for future use

#Calculating size
size <- log(market_cap_monthly_merged)
names(size) <- gsub(".Adjusted", ".Size", unlist(lapply(daily_adj_close, names))) #New names
size <- as.xts(apply(size, 2, function(x) {ifelse(is.finite(x), x, NA)}), order.by = index(size)) #Replace -Inf with NA
size <- strip_empty_lines(size) #Remove empty rows
#saveRDS(size, file = "Final data/size.RData") #Saving for future use

We also extract the factors Momentum and Liquidity which are essential for the calculation of $\alpha$.

In [59]:
#Extracting momentum
momentum <- as.data.frame(momentum)
colnames(momentum)[1] <- "Date"
momentum$Date <-  as.Date(paste0(as.character(momentum$Date),"01"), "%Y%m%d") + months(1) - days(1) #Creating a date (end of month)
momentum_xts <- xts(momentum[, 2], order.by = momentum$Date) #Converting to xts
names(momentum_xts) <- "Momentum" #Renaming
momentum_xts <- momentum_xts["2007/"] #Restricting the period
#saveRDS(momentum_xts, "Final data/momentum.RData") #Saving for future use

#Extracting liquidity
colnames(liquidity)[1] <- "Date"
liquidity$Date <-  as.Date(paste0(as.character(liquidity$Date),"01"), "%Y%m%d") + months(1) - days(1) #Creating a date (end of month)
liquidity_xts <- xts(liquidity[, 4], order.by = liquidity$Date) #Converting to xts
names(liquidity_xts) <- "Liquidity" #Renaming
liquidity_xts <- liquidity_xts["2007/"] #Restricting the period
#saveRDS(liquidity_xts, "Final data/liquidity.RData") #Saving for future use

### Appendix B

#### List of stocks
<a id='appendix_b'></a> 

Bellow we provide the list of stocks (their tickers) that were used for the analysis.

In [67]:
stock_names <- c(rep(NA, ncol(monthly_returns))) #empty vector to hold names
for (i in 1:ncol(monthly_returns)) { #loop through the return columns
  stock_names[i] <- sub(pattern = ".MonthlyReturns", replacement = "", x = names(monthly_returns[,i])) #strip name of each column, remove second part
}
matrix(stock_names, ncol = 10) #print names

[,1] [,2] [,3] [,4] [,5] [,6]  [,7] [,8] [,9] [,10]
 [1,] MDLZ DG   CSX  CF   USB  V     PAYX NSC  SPG  HRL  
 [2,] NRG  SBUX BMY  TER  PRU  AVB   AMGN RHI  MAS  KEY  
 [3,] IT   AMAT TROW MMM  CE   DXCM  ACN  AON  TFC  DVN  
 [4,] NTAP STT  TEL  NEM  F    SEE   TYL  MS   ADI  RE   
 [5,] XRAY T    DGX  MA   RL   GNRC  RCL  ILMN LDOS JPM  
 [6,] MPWR NOC  ANSS HOLX IRM  PG    WYNN MAA  EFX  MO   
 [7,] AMP  ED   MTCH EW   JNPR SIVB  GS   MAR  AAPL MSI  
 [8,] LMT  CLX  BKNG A    FFIV VFC   VRSK ESS  RTX  FTNT 
 [9,] URI  ULTA MGM  APA  BXP  CDNS  WM   EBAY BWA  SYY  
[10,] LKQ  AAL  WFC  ANTM ADSK CTXS  ECL  FE   PKI  GOOGL
[11,] PNR  ES   TAP  GL   D    SCHW  PH   HPQ  WELL MDT  
[12,] FRT  WAT  SWKS NFLX AEE  ATO   IVZ  KMB  GILD DE   
[13,] EA   PPL  BDX  TT   NTRS DISCK UAA  FCX  KIM  DRE  
[14,] CCI  TSCO MCHP AME  FDS  BLK   MRK  CRM  ITW  ABC  
[15,] RF   AES  BR   NVR  PNW  VMC   EMN  XOM  EOG  VTRS 
[16,] WEC  EXPE TRMB WMB  HD   EL    CVX  MOH  KR   WDC  
[17,] WY   ZBH  CBRE HES  MU   ROST  EMR  IDXX PGR  AZO  
[18,] UPS  TRV  DUK  NDAQ CMI  EQIX  AMZN WST  OMC  BIIB 
[19,] O    DRI  POOL NWL  ALK  TMUS  SBNY ETN  IEX  PARA 
[20,] SBAC BRO  DFS  PEP  PWR  AXP   HAS  AEP  HIG  AMT  
[21,] ISRG NKE  PVH  CB   CAH  FDX   EIX  ALGN UNH  ABT  
[22,] REG  FITB LUMN ALL  KO   INTC  UNP  REGN MTB  ROK  
[23,] PM   CMA  INTU PPG  DAL  AAP   CHD  SLB  TDG  DIS  
[24,] CSCO ARE  HSIC VZ   PHM  LOW   VRTX PFE  SRE  AVY  
[25,] MRO  LRCX GPN  CNP  NDSN AFL   RSG  HON  WHR  DTE

## References

* Bali, Turan G., Robert F. Engle, and Scott Murray. Empirical asset pricing: The cross section of stock returns. John Wiley & Sons, 2016.